In [1]:
using Pipe
# using MarketTechnicals
# using Plots
# using TimeSeries
using DataFrames
# using StatsPlots
using CSV
using RollingFunctions
using Query
# using GLMakie

using JSON
using DataFrames
using Dates
using Colors
using ColorSchemes
using Indicators

import Colors
using Plots
using Interact

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-8244136837299451535\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-11773535511231773233\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-8244136837299451535\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
json_data_folder = "../user_data/data/binance"
json_filename = "BTC_USDT-15m.json"
json_filepath = "$json_data_folder/$json_filename"
f = open(json_filepath, "r")
json_data = JSON.parse(f)
close(f)

In [7]:
dfa = DataFrame(unix_time = Int64[], open=Float64[], high=Float64[], low=Float64[], close=Float64[], volume=Float64[])
for r in json_data push!(dfa, r) end
dfa.unix_time .= dfa.unix_time ./1000
# dfa.time .= unix2datetime.(dfa.unix_time)

insertcols!(dfa, :time => unix2datetime.(dfa.unix_time))
nrow(dfa)

13939

In [187]:
start_date = DateTime(2021, 1, 1)
end_date = DateTime(2021, 5, 16)
vdf = dfa[(dfa.time .>= start_date) .& (dfa.time .<= end_date) , :]
ut, tt, hh, ll, cc = vdf.unix_time, vdf.time, vdf.high, vdf.low, vdf.close
# h14max = [fill(vdf[1,:high],8-1)...,rollmax(vdf.high, 8)...]
#  length(h14max), length(vdf.close)
# hlc = [hh ll cc]
length(tt)

12922

(Tuple{Int64, Float64}[], Tuple{Int64, Float64}[], Tuple{Int64, Float64}[], Tuple{Int64, Float64}[], Tuple{Int64, Float64}[], Tuple{Int64, Float64}[], Tuple{Int64, Float64}[])

In [169]:
ri(a,i) = a[length(a) - i + 1]
rip(a,i) = ri(a,i)[2]
rit(a,i) = ri(a,i)[1]

PricePoint = Tuple{Int64,Float64}
qp, qh, ql, qll, qhl, qhh, qlh = PricePoint[], PricePoint[], PricePoint[], PricePoint[], 
    PricePoint[], PricePoint[], PricePoint[]

pushq(t, p) = begin
#     push!(qp,(t,p))
    push!(qp, (t,p)) 
    if length(qp) >= 3
        qprv = qp[length(qp)-1]
        up = true
        if qp[length(qp)-2][2] > qp[length(qp)-1][2] && qp[length(qp)-1][2] < qp[length(qp)][2]
            push!(ql, qprv )            
            if length(ql) > 3                
                qlprv = ql[length(ql)-1]
                if ql[length(ql)-2][2] > ql[length(ql)-1][2] && ql[length(ql)-1][2] < ql[length(ql)][2]
                    push!(qll, qlprv )
                end
                if ql[length(ql)-2][2] < ql[length(ql)-1][2] && ql[length(ql)-1][2] > ql[length(ql)][2]
                    push!(qhl, qlprv )
                end
            end
        end
        
        if qp[length(qp)-2][2] < qp[length(qp)-1][2] && qp[length(qp)-1][2] > qp[length(qp)][2]
            push!(qh, qprv )
            if length(qh) > 3
                qhprv = qh[length(qh)-1]
                if qh[length(qh)-2][2] > qh[length(qh)-1][2] && qh[length(qh)-1][2] < qh[length(qh)][2]
                    push!(qlh, qhprv )
                end
                if qh[length(qh)-2][2] < qh[length(qh)-1][2] && qh[length(qh)-1][2] > qh[length(qh)][2]
                    push!(qhh, qhprv )
                end
            end
        end
    end
    length(qp), length(qh), length(ql),length(qhh), length(qlh), length(qhl), length(qll)
end

pushq (generic function with 1 method)

In [220]:
qp, qh, ql, qll, qhl, qhh, qlh = PricePoint[], PricePoint[], PricePoint[], PricePoint[], 
    PricePoint[], PricePoint[], PricePoint[]
ar = cc

ema = Indicators.ema
ema26 = ema(ar,n=26)
ema50 = ema(ar,n=50)
ema100 = ema(ar,n=100)
ema200 = ema(ar,n=200)
sma = Indicators.sma
sma26 = sma(ar,n=26)
sma50 = sma(ar,n=50)
sma100 = sma(ar,n=100)
sma200 = sma(ar,n=200)

for i in 1:length(ar)   
    pushq(ut[i], ar[i])
end

In [ ]:
buy_sigs = []
for i in 1:length(ar)
    if i < 30 || length(qhl) < 1 || length(qhh) < 1 continue end

    if rip(ql,1) < rip(qhl,1) && rip(qh,1) > rip(qhl,1) && rip(qp,1) < rip(qp,2) &&
        rip(qp,1) < 1.01 * rip(qhl,1) && rip(qp,1) < rip(qhh,1)
        push!(buy_sigs, ut[i])
    end
end

In [192]:

ar = cc
utstep = 500
nsteps = Int(trunc(length(ar) / utstep * 2))

51

In [234]:
# string(xtt[1])

In [250]:

# xsteps(a) = vcat([[a[i-1][1],  a[i][1], a[i][1], NaN] for i in 2:length(a)]...)
# ysteps(a) = vcat([[a[i-1][2],  a[i-1][2],  a[i][2], NaN] for i in 2:length(a)]...)
# steps!(a;args...) = plot!(xsteps(a), ysteps(a);args...)

ar = cc
utstep = 700
nsteps = Int(trunc(length(ar) / utstep * 10))

mp = @manipulate for 
            stput in slider(1:nsteps; label="start", default=1),
            utstep in slider(100:3000; label="zoom", default=1)
    
    sut = Int(trunc(stput * utstep / 10))
    eut = min(sut + utstep, length(ar))

    xut = ut[sut:eut]
    xtt = tt[sut:eut]
    xar = ar[sut:eut]
    xema26= ema26[sut:eut]
    xema50= ema50[sut:eut]
    xema100= ema100[sut:eut]
    xema200= ema200[sut:eut]
    xsma26= sma26[sut:eut]
    xsma50= sma50[sut:eut]
    xsma100= sma100[sut:eut]
    xsma200= sma200[sut:eut]


    ftt = Dates.format.(xtt,"yyyy-mm-dd HH:MM");
    
#     length(xtt) > 0 && latex(xtt[1])
    
    f = plot(xut, xar, size=(1200,400), color=:blue, lw=0.5, legend = :outertopright, labels="close", 
        xticks=(xut[1:max(1,Int(ceil(length(xut) / 20))):length(xut)],ftt), xrot=60)
    # plot!(qh)


    plot!(xut, xema100, fillrange=xema100*0.97, fillalpha=0.2)
    plot!(xut, xema100, fillrange=xema100*1.03, fillalpha=0.2)
    
#     plot!(xut, xema26)
#     plot!(xut, xema50)
#     plot!(xut, xema100)
#     plot!(xut, xema200)
    
    
#     plot!(xut, xsma26)
#     plot!(xut, xsma50)
#     plot!(xut, xsma100)
#     plot!(xut, xsma200)
    
    
#     plot!(xut, xar, fillrange=xar*1.01, fillalpha=0.2)
#     plot!(xut, xar, fillrange=xar*0.99, fillalpha=0.2)
    
    # steps!(qll)
    # plot!(xsteps(qll), ysteps(qll), fillrange=ysteps(qll)*1.01, fillalpha=0.1)
    # steps!(qhl)
    # steps!(qhh)
    # steps!(qlh)
    # steps!(qhh)

    # plot!(xsteps(qhh), ysteps(qhh), fillrange=ysteps(qhh)*0.995, fillalpha=0.2)
#     scatter!(qll[sut:eut], markersize=2)
#     scatter!(qhh[sut:eut], markersize=2)
    # scatter!(qlh)
    # plot!(qlh)


    # length(buy_sigs) > 0 && vline!(buy_sigs, lw=0.4)



#     f
end
#     using Blink
#     w = Window()
    body!(w, mp);
mp

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["start"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 184, :default => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :min => 1, :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "index" => (Observable{Int64} with 2 listeners. Value:
92, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/johnsmith/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/all.js"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/InteractBase/Zfu5P/src/../assets/style.css"), Asset("css", nothing, "/Users/johnsmith/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000014b9cecb0, Task (runnable) @0x000000014b9cecb0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"35\"